In [1]:
# Batch test for Direct Classification benchmark 4 (cropped square & data augmentation images, normalizing with mean and std)
import torch, os, open_clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
from torchvision.transforms import ToTensor
import albumentations as A


class AlbumentationsTransform2:
    def __init__(self, transform=None):
        self.transform = transform

    def __call__(self, img):
        img = np.array(img)  # Convert to numpy array
        img = self.transform(image=img)['image']
        return Image.fromarray(img)  # Convert back to PIL image

# Define the CLAHE transformation
clahe = A.CLAHE(p=1.0, clip_limit=6.0, tile_grid_size=(12, 12))
CLAHE = AlbumentationsTransform2(clahe) # normalizing using Adaptive Histogram Equalization (CLAHE)

caption_map = {
    "healthy": "healthy normal",
    "doubtful": "doubtful osteoarthritis",
    "minimal": "minimal osteoarthritis",
    "moderate": "moderate osteoarthritis",
    "severe": "severe osteoarthritis"
}

classification = list(caption_map.values())

def get_ground_truth(filename, caption_map):
    for keyword, ground_truth in caption_map.items():
        if keyword in filename:
            return ground_truth
    return None

def process_single_epoch(pretrained_model, sentences):
    img_dir = '/mnt/g/Datasets/Knee_OA/Original/test/images/'
    model, _, preprocess = open_clip.create_model_and_transforms(
        model_name="coca_ViT-L-14",
        pretrained=pretrained_model,
    )
    pt_dir = os.path.dirname(pretrained_model)
    epoch_number = os.path.basename(pretrained_model).split('epoch_')[1].split('.pt')[0]
    df_all = pd.DataFrame()

    # Prepare the tokenizer for sentences
    tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')
    text = tokenizer(sentences)
    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = model.encode_text(text)
        text_features /= text_features.norm(dim=-1, keepdim=True)

    def read_single_image(img_dir, img_filename, sentences, text_features):
        #get the ground truth value
        image_path = os.path.join(img_dir, img_filename)
        ground_truth = get_ground_truth(img_filename, caption_map)
        
        # load a sample image
        image = preprocess(Image.open(image_path).convert('RGB')).unsqueeze(0)
        
        with torch.no_grad(), torch.cuda.amp.autocast():
            image_features = model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            text_probs = text_probs.cpu().tolist()[0]

        # Construct the dictionary
        class_predict_dict = dict(zip(sentences, text_probs))
        # Extract the key with the largest value
        class_predict = max(class_predict_dict, key=class_predict_dict.get)
        
        if ground_truth == class_predict:
            predicted_correct = 1
        else:
            predicted_correct = 0
        
        pred_dict = {'img_filename': [img_filename], 'class_predict_dict': [class_predict_dict], 'ground_truth': [ground_truth], 'class_predict': [class_predict], 'predicted_correct': [predicted_correct]}
        df_single = pd.DataFrame(pred_dict)
        
        return df_single

    all_images = os.listdir(img_dir)
    for img_filename in tqdm(all_images, desc=f"epoch_{epoch_number}: Processing images", unit="image"):
        df = read_single_image(img_dir, img_filename, sentences, text_features)
        df_all = pd.concat([df_all, df]).reset_index(drop=True)

    #save df_all to csv
    csv_file = f"/home/yilu/Development/open_clip/Knee_OA/temp/epoch_{epoch_number}.csv"
    df_all.to_csv(csv_file, index=False)

    total_rows = len(df_all)
    # Count the occurrences of '1' in the 'predicted_correct' column
    count_ones = df_all['predicted_correct'].sum()
    epoch_filename = os.path.splitext(os.path.basename(pretrained_model))[0]
    predicted_correct_pct = count_ones /total_rows *100
    print(f"Epoch_file: {epoch_filename}. Total images: {total_rows}, predicted_correct': {count_ones}, predicted_correct (%)': {predicted_correct_pct}%")
    
    epoch_dict = {'pt_dir': [pt_dir],
                'epoch_filename': [epoch_filename], 
                'sentences': [sentences], 
                'total_rows': [total_rows], 
                'predicted_correct': [count_ones], 
                'predicted_correct (%)': [predicted_correct_pct]}
    df_single_epoch = pd.DataFrame(epoch_dict)
    return df_single_epoch


def process_single_pretrained_model(pretrained_model, csv_file):
    df_single_pretrained_model = pd.DataFrame()
    df = process_single_epoch(pretrained_model, classification)
    df_single_pretrained_model = pd.concat([df_single_pretrained_model, df]).reset_index(drop=True)
    save_to_csv(df_single_pretrained_model, csv_file)
    
def extract_epoch_num(filepath):
    # Split the filename from the path
    filename = os.path.basename(filepath) #filepath.split('/')[-1]
    # Extract the number between "epoch_" and ".pt"
    epoch_num = int(filename.split('epoch_')[1].split('.pt')[0])
    return epoch_num

def save_to_csv(df, csv_file):
    if not os.path.exists(csv_file):
        df.to_csv(csv_file, index=False)
    else:
        # If the CSV file already exists, append without header
        df.to_csv(csv_file, mode='a', header=False, index=False)

def main():
    #obtain the latest pretrained model folder
    pt_folder_root = '/mnt/g/Logtemp/open_clip/Knee_OA/'
    dir_1 = [a for a in os.listdir(pt_folder_root) if a.startswith('2023_08_24-23')][0]
    pt_folder_path = os.path.join(pt_folder_root, dir_1, 'checkpoints')

    pt_files = [os.path.join(pt_folder_path, filename) for filename in os.listdir(pt_folder_path) if filename.endswith('.pt')]
    #filter pt_files with low epoch number
    pt_files = [pt_file for pt_file in pt_files if extract_epoch_num(pt_file) >= 0]
    # Sort the list
    sorted_pt_files = sorted(pt_files, key=extract_epoch_num)
    csv_file = "/home/yilu/Development/open_clip/Knee_OA/temp/benchmark_all_epochs.csv"
    for pretrained_model in sorted_pt_files:
        process_single_pretrained_model(pretrained_model, csv_file)


main()

epoch_4: Processing images: 100%|██████████| 1656/1656 [14:15<00:00,  1.94image/s]


Epoch_file: epoch_4. Total images: 1656, predicted_correct': 456, predicted_correct (%)': 27.536231884057973%


epoch_8: Processing images: 100%|██████████| 1656/1656 [13:32<00:00,  2.04image/s]


Epoch_file: epoch_8. Total images: 1656, predicted_correct': 797, predicted_correct (%)': 48.128019323671495%


epoch_12: Processing images: 100%|██████████| 1656/1656 [13:05<00:00,  2.11image/s]


...

Epoch_file: epoch_168. Total images: 1656, predicted_correct': 1121, predicted_correct (%)': 67.69323671497585%


epoch_172: Processing images: 100%|██████████| 1656/1656 [14:11<00:00,  1.94image/s]


Epoch_file: epoch_172. Total images: 1656, predicted_correct': 1091, predicted_correct (%)': 65.88164251207729%


epoch_176: Processing images:  11%|█         | 176/1656 [01:32<12:59,  1.90image/s]


KeyboardInterrupt: 

In [3]:
# Batch test for Direct Classification benchmark 5 (cropped square & data augmentation images, normalizing with mean and std)
import torch, os, open_clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
from torchvision.transforms import ToTensor
import albumentations as A


class AlbumentationsTransform2:
    def __init__(self, transform=None):
        self.transform = transform

    def __call__(self, img):
        img = np.array(img)  # Convert to numpy array
        img = self.transform(image=img)['image']
        return Image.fromarray(img)  # Convert back to PIL image

# Define the CLAHE transformation
clahe = A.CLAHE(p=1.0, clip_limit=6.0, tile_grid_size=(12, 12))
CLAHE = AlbumentationsTransform2(clahe) # normalizing using Adaptive Histogram Equalization (CLAHE)

caption_map = {
    "healthy": "healthy normal",
    "doubtful": "doubtful osteoarthritis",
    "minimal": "minimal osteoarthritis",
    "moderate": "moderate osteoarthritis",
    "severe": "severe osteoarthritis"
}

classification = list(caption_map.values())

def get_ground_truth(filename, caption_map):
    for keyword, ground_truth in caption_map.items():
        if keyword in filename:
            return ground_truth
    return None

def process_single_epoch(pretrained_model, sentences):
    img_dir = '/mnt/g/Datasets/Knee_OA/Original/test/images/'
    model, _, preprocess = open_clip.create_model_and_transforms(
        model_name="coca_ViT-L-14",
        pretrained=pretrained_model,
    )
    pt_dir = os.path.dirname(pretrained_model)
    epoch_number = os.path.basename(pretrained_model).split('epoch_')[1].split('.pt')[0]
    df_all = pd.DataFrame()

    # Prepare the tokenizer for sentences
    tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')
    text = tokenizer(sentences)
    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = model.encode_text(text)
        text_features /= text_features.norm(dim=-1, keepdim=True)

    def read_single_image(img_dir, img_filename, sentences, text_features):
        #get the ground truth value
        image_path = os.path.join(img_dir, img_filename)
        ground_truth = get_ground_truth(img_filename, caption_map)
        
        # load a sample image
        image = preprocess(Image.open(image_path).convert('RGB')).unsqueeze(0)
        
        with torch.no_grad(), torch.cuda.amp.autocast():
            image_features = model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            text_probs = text_probs.cpu().tolist()[0]

        # Construct the dictionary
        class_predict_dict = dict(zip(sentences, text_probs))
        # Extract the key with the largest value
        class_predict = max(class_predict_dict, key=class_predict_dict.get)
        
        if ground_truth == class_predict:
            predicted_correct = 1
        else:
            predicted_correct = 0
        
        pred_dict = {'img_filename': [img_filename], 'class_predict_dict': [class_predict_dict], 'ground_truth': [ground_truth], 'class_predict': [class_predict], 'predicted_correct': [predicted_correct]}
        df_single = pd.DataFrame(pred_dict)
        
        return df_single

    all_images = os.listdir(img_dir)
    for img_filename in tqdm(all_images, desc=f"epoch_{epoch_number}: Processing images", unit="image"):
        df = read_single_image(img_dir, img_filename, sentences, text_features)
        df_all = pd.concat([df_all, df]).reset_index(drop=True)

    #save df_all to csv
    csv_file = f"/home/yilu/Development/open_clip/Knee_OA/temp1/epoch_{epoch_number}.csv"
    df_all.to_csv(csv_file, index=False)

    total_rows = len(df_all)
    # Count the occurrences of '1' in the 'predicted_correct' column
    count_ones = df_all['predicted_correct'].sum()
    epoch_filename = os.path.splitext(os.path.basename(pretrained_model))[0]
    predicted_correct_pct = count_ones /total_rows *100
    print(f"Epoch_file: {epoch_filename}. Total images: {total_rows}, predicted_correct': {count_ones}, predicted_correct (%)': {predicted_correct_pct}%")
    
    epoch_dict = {'pt_dir': [pt_dir],
                'epoch_filename': [epoch_filename], 
                'sentences': [sentences], 
                'total_rows': [total_rows], 
                'predicted_correct': [count_ones], 
                'predicted_correct (%)': [predicted_correct_pct]}
    df_single_epoch = pd.DataFrame(epoch_dict)
    return df_single_epoch


def process_single_pretrained_model(pretrained_model, csv_file):
    df_single_pretrained_model = pd.DataFrame()
    df = process_single_epoch(pretrained_model, classification)
    df_single_pretrained_model = pd.concat([df_single_pretrained_model, df]).reset_index(drop=True)
    save_to_csv(df_single_pretrained_model, csv_file)
    
def extract_epoch_num(filepath):
    # Split the filename from the path
    filename = os.path.basename(filepath) #filepath.split('/')[-1]
    # Extract the number between "epoch_" and ".pt"
    epoch_num = int(filename.split('epoch_')[1].split('.pt')[0])
    return epoch_num

def save_to_csv(df, csv_file):
    if not os.path.exists(csv_file):
        df.to_csv(csv_file, index=False)
    else:
        # If the CSV file already exists, append without header
        df.to_csv(csv_file, mode='a', header=False, index=False)

def main():
    #obtain the latest pretrained model folder
    pt_folder_root = '/mnt/g/Logtemp/open_clip/Knee_OA/'
    dir_1 = [a for a in os.listdir(pt_folder_root) if a.startswith('2023_08_25-18_55_21')][0]
    pt_folder_path = os.path.join(pt_folder_root, dir_1, 'checkpoints')

    pt_files = [os.path.join(pt_folder_path, filename) for filename in os.listdir(pt_folder_path) if filename.endswith('.pt')]
    #filter pt_files with low epoch number
    pt_files = [pt_file for pt_file in pt_files if extract_epoch_num(pt_file) >= 193]
    # Sort the list
    sorted_pt_files = sorted(pt_files, key=extract_epoch_num)
    csv_file = "/home/yilu/Development/open_clip/Knee_OA/temp1/benchmark_all_epochs.csv"
    for pretrained_model in sorted_pt_files:
        process_single_pretrained_model(pretrained_model, csv_file)


main()

epoch_196: Processing images: 100%|██████████| 1656/1656 [12:41<00:00,  2.17image/s]


Epoch_file: epoch_196. Total images: 1656, predicted_correct': 1123, predicted_correct (%)': 67.81400966183575%


epoch_200: Processing images: 100%|██████████| 1656/1656 [12:43<00:00,  2.17image/s]


Epoch_file: epoch_200. Total images: 1656, predicted_correct': 1129, predicted_correct (%)': 68.17632850241546%


epoch_204: Processing images: 100%|██████████| 1656/1656 [12:24<00:00,  2.22image/s]


...

Epoch_file: epoch_268. Total images: 1656, predicted_correct': 1124, predicted_correct (%)': 67.8743961352657%


epoch_272: Processing images: 100%|██████████| 1656/1656 [13:50<00:00,  1.99image/s]


Epoch_file: epoch_272. Total images: 1656, predicted_correct': 1139, predicted_correct (%)': 68.78019323671496%


epoch_276: Processing images:  14%|█▍        | 239/1656 [02:04<12:20,  1.91image/s]


KeyboardInterrupt: 

In [4]:
# Batch test for Direct Classification benchmark 6 (cropped square & data augmentation images, starting to normalize with mean and std)
import torch, os, open_clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
from torchvision.transforms import ToTensor
import albumentations as A

caption_map = {
    "healthy": "healthy normal",
    "doubtful": "doubtful osteoarthritis",
    "minimal": "minimal osteoarthritis",
    "moderate": "moderate osteoarthritis",
    "severe": "severe osteoarthritis"
}

classification = list(caption_map.values())

def get_ground_truth(filename, caption_map):
    for keyword, ground_truth in caption_map.items():
        if keyword in filename:
            return ground_truth
    return None

def process_single_epoch(pretrained_model, sentences):
    img_dir = '/mnt/g/Datasets/Knee_OA/Original/test/images/'
    model, _, preprocess = open_clip.create_model_and_transforms(
        model_name="coca_ViT-L-14",
        pretrained=pretrained_model,
        image_mean = (0.60623126, 0.60623126, 0.60623126),
        image_std = (0.19362484, 0.19362484, 0.19362484),
        # color_image = False,
    )
    pt_dir = os.path.dirname(pretrained_model)
    epoch_number = os.path.basename(pretrained_model).split('epoch_')[1].split('.pt')[0]
    df_all = pd.DataFrame()

    # Prepare the tokenizer for sentences
    tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')
    text = tokenizer(sentences)
    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = model.encode_text(text)
        text_features /= text_features.norm(dim=-1, keepdim=True)

    def read_single_image(img_dir, img_filename, sentences, text_features):
        #get the ground truth value
        image_path = os.path.join(img_dir, img_filename)
        ground_truth = get_ground_truth(img_filename, caption_map)
        
        # load a sample image
        image = preprocess(Image.open(image_path).convert('RGB')).unsqueeze(0)
        
        with torch.no_grad(), torch.cuda.amp.autocast():
            image_features = model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            text_probs = text_probs.cpu().tolist()[0]

        # Construct the dictionary
        class_predict_dict = dict(zip(sentences, text_probs))
        # Extract the key with the largest value
        class_predict = max(class_predict_dict, key=class_predict_dict.get)
        
        if ground_truth == class_predict:
            predicted_correct = 1
        else:
            predicted_correct = 0
        
        pred_dict = {'img_filename': [img_filename], 'class_predict_dict': [class_predict_dict], 'ground_truth': [ground_truth], 'class_predict': [class_predict], 'predicted_correct': [predicted_correct]}
        df_single = pd.DataFrame(pred_dict)
        
        return df_single

    all_images = os.listdir(img_dir)
    for img_filename in tqdm(all_images, desc=f"epoch_{epoch_number}: Processing images", unit="image"):
        df = read_single_image(img_dir, img_filename, sentences, text_features)
        df_all = pd.concat([df_all, df]).reset_index(drop=True)

    #save df_all to csv
    csv_file = f"/home/yilu/Development/open_clip/Knee_OA/temp2/epoch_{epoch_number}.csv"
    df_all.to_csv(csv_file, index=False)

    total_rows = len(df_all)
    # Count the occurrences of '1' in the 'predicted_correct' column
    count_ones = df_all['predicted_correct'].sum()
    epoch_filename = os.path.splitext(os.path.basename(pretrained_model))[0]
    predicted_correct_pct = count_ones /total_rows *100
    print(f"Epoch_file: {epoch_filename}. Total images: {total_rows}, predicted_correct': {count_ones}, predicted_correct (%)': {predicted_correct_pct}%")
    
    epoch_dict = {'pt_dir': [pt_dir],
                'epoch_filename': [epoch_filename], 
                'sentences': [sentences], 
                'total_rows': [total_rows], 
                'predicted_correct': [count_ones], 
                'predicted_correct (%)': [predicted_correct_pct]}
    df_single_epoch = pd.DataFrame(epoch_dict)
    return df_single_epoch


def process_single_pretrained_model(pretrained_model, csv_file):
    df_single_pretrained_model = pd.DataFrame()
    df = process_single_epoch(pretrained_model, classification)
    df_single_pretrained_model = pd.concat([df_single_pretrained_model, df]).reset_index(drop=True)
    save_to_csv(df_single_pretrained_model, csv_file)
    
def extract_epoch_num(filepath):
    # Split the filename from the path
    filename = os.path.basename(filepath) #filepath.split('/')[-1]
    # Extract the number between "epoch_" and ".pt"
    epoch_num = int(filename.split('epoch_')[1].split('.pt')[0])
    return epoch_num

def save_to_csv(df, csv_file):
    if not os.path.exists(csv_file):
        df.to_csv(csv_file, index=False)
    else:
        # If the CSV file already exists, append without header
        df.to_csv(csv_file, mode='a', header=False, index=False)

def main():
    #obtain the latest pretrained model folder
    pt_folder_root = '/mnt/g/Logtemp/open_clip/Knee_OA/'
    dir_1 = [a for a in os.listdir(pt_folder_root) if a.startswith('2023_08_27-00')][0]
    pt_folder_path = os.path.join(pt_folder_root, dir_1, 'checkpoints')

    pt_files = [os.path.join(pt_folder_path, filename) for filename in os.listdir(pt_folder_path) if filename.endswith('.pt')]
    #filter pt_files with low epoch number
    pt_files = [pt_file for pt_file in pt_files if extract_epoch_num(pt_file) >= 112]
    # Sort the list
    sorted_pt_files = sorted(pt_files, key=extract_epoch_num)
    csv_file = "/home/yilu/Development/open_clip/Knee_OA/temp2/benchmark_all_epochs.csv"
    for pretrained_model in sorted_pt_files:
        process_single_pretrained_model(pretrained_model, csv_file)


main()

epoch_112: Processing images: 100%|██████████| 1656/1656 [14:51<00:00,  1.86image/s]


Epoch_file: epoch_112. Total images: 1656, predicted_correct': 1120, predicted_correct (%)': 67.6328502415459%


epoch_116: Processing images: 100%|██████████| 1656/1656 [14:11<00:00,  1.94image/s]


Epoch_file: epoch_116. Total images: 1656, predicted_correct': 1131, predicted_correct (%)': 68.29710144927536%


epoch_120: Processing images: 100%|██████████| 1656/1656 [13:28<00:00,  2.05image/s]


...

Epoch_file: epoch_384. Total images: 1656, predicted_correct': 1137, predicted_correct (%)': 68.65942028985508%


epoch_388: Processing images: 100%|██████████| 1656/1656 [13:42<00:00,  2.01image/s]


Epoch_file: epoch_388. Total images: 1656, predicted_correct': 1138, predicted_correct (%)': 68.71980676328504%


epoch_392: Processing images: 100%|██████████| 1656/1656 [13:43<00:00,  2.01image/s]

Epoch_file: epoch_392. Total images: 1656, predicted_correct': 1127, predicted_correct (%)': 68.05555555555556%


In [6]:
# Batch test for Direct Classification benchmark 6 (cropped square & data augmentation images, starting to normalize with mean and std, no resizing)
import torch, os, open_clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
from torchvision.transforms import ToTensor
import albumentations as A

caption_map = {
    "healthy": "healthy normal",
    "doubtful": "doubtful osteoarthritis",
    "minimal": "minimal osteoarthritis",
    "moderate": "moderate osteoarthritis",
    "severe": "severe osteoarthritis"
}

classification = list(caption_map.values())

def get_ground_truth(filename, caption_map):
    for keyword, ground_truth in caption_map.items():
        if keyword in filename:
            return ground_truth
    return None

def process_single_epoch(pretrained_model, sentences):
    img_dir = '/mnt/g/Datasets/Knee_OA/Original/test/images/'
    model, _, preprocess = open_clip.create_model_and_transforms(
        model_name="coca_ViT-L-14",
        pretrained=pretrained_model,
        image_mean = (0.60623126, 0.60623126, 0.60623126),
        image_std = (0.19362484, 0.19362484, 0.19362484),
        # color_image = False,
    )
    pt_dir = os.path.dirname(pretrained_model)
    epoch_number = os.path.basename(pretrained_model).split('epoch_')[1].split('.pt')[0]
    df_all = pd.DataFrame()

    # Prepare the tokenizer for sentences
    tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')
    text = tokenizer(sentences)
    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = model.encode_text(text)
        text_features /= text_features.norm(dim=-1, keepdim=True)

    def read_single_image(img_dir, img_filename, sentences, text_features):
        #get the ground truth value
        image_path = os.path.join(img_dir, img_filename)
        ground_truth = get_ground_truth(img_filename, caption_map)
        
        # load a sample image
        image = preprocess(Image.open(image_path).convert('RGB')).unsqueeze(0)
        
        with torch.no_grad(), torch.cuda.amp.autocast():
            image_features = model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            text_probs = text_probs.cpu().tolist()[0]

        # Construct the dictionary
        class_predict_dict = dict(zip(sentences, text_probs))
        # Extract the key with the largest value
        class_predict = max(class_predict_dict, key=class_predict_dict.get)
        
        if ground_truth == class_predict:
            predicted_correct = 1
        else:
            predicted_correct = 0
        
        pred_dict = {'img_filename': [img_filename], 'class_predict_dict': [class_predict_dict], 'ground_truth': [ground_truth], 'class_predict': [class_predict], 'predicted_correct': [predicted_correct]}
        df_single = pd.DataFrame(pred_dict)
        
        return df_single

    all_images = os.listdir(img_dir)
    for img_filename in tqdm(all_images, desc=f"epoch_{epoch_number}: Processing images", unit="image"):
        df = read_single_image(img_dir, img_filename, sentences, text_features)
        df_all = pd.concat([df_all, df]).reset_index(drop=True)

    #save df_all to csv
    csv_file = f"/home/yilu/Development/open_clip/Knee_OA/temp3/epoch_{epoch_number}.csv"
    df_all.to_csv(csv_file, index=False)

    total_rows = len(df_all)
    # Count the occurrences of '1' in the 'predicted_correct' column
    count_ones = df_all['predicted_correct'].sum()
    epoch_filename = os.path.splitext(os.path.basename(pretrained_model))[0]
    predicted_correct_pct = count_ones /total_rows *100
    print(f"Epoch_file: {epoch_filename}. Total images: {total_rows}, predicted_correct': {count_ones}, predicted_correct (%)': {predicted_correct_pct}%")
    
    epoch_dict = {'pt_dir': [pt_dir],
                'epoch_filename': [epoch_filename], 
                'sentences': [sentences], 
                'total_rows': [total_rows], 
                'predicted_correct': [count_ones], 
                'predicted_correct (%)': [predicted_correct_pct]}
    df_single_epoch = pd.DataFrame(epoch_dict)
    return df_single_epoch


def process_single_pretrained_model(pretrained_model, csv_file):
    df_single_pretrained_model = pd.DataFrame()
    df = process_single_epoch(pretrained_model, classification)
    df_single_pretrained_model = pd.concat([df_single_pretrained_model, df]).reset_index(drop=True)
    save_to_csv(df_single_pretrained_model, csv_file)
    
def extract_epoch_num(filepath):
    # Split the filename from the path
    filename = os.path.basename(filepath) #filepath.split('/')[-1]
    # Extract the number between "epoch_" and ".pt"
    epoch_num = int(filename.split('epoch_')[1].split('.pt')[0])
    return epoch_num

def save_to_csv(df, csv_file):
    if not os.path.exists(csv_file):
        df.to_csv(csv_file, index=False)
    else:
        # If the CSV file already exists, append without header
        df.to_csv(csv_file, mode='a', header=False, index=False)

def main():
    #obtain the latest pretrained model folder
    pt_folder_root = '/mnt/g/Logtemp/open_clip/Knee_OA/'
    dir_1 = [a for a in os.listdir(pt_folder_root) if a.startswith('2023_08_27-21')][0]
    pt_folder_path = os.path.join(pt_folder_root, dir_1, 'checkpoints')

    pt_files = [os.path.join(pt_folder_path, filename) for filename in os.listdir(pt_folder_path) if filename.endswith('.pt')]
    #filter pt_files with low epoch number
    pt_files = [pt_file for pt_file in pt_files if extract_epoch_num(pt_file) >= 0]
    # Sort the list
    sorted_pt_files = sorted(pt_files, key=extract_epoch_num, reverse=True)
    csv_file = "/home/yilu/Development/open_clip/Knee_OA/temp3/benchmark_all_epochs.csv"
    for pretrained_model in sorted_pt_files:
        process_single_pretrained_model(pretrained_model, csv_file)


main()

epoch_400: Processing images: 100%|██████████| 1656/1656 [13:36<00:00,  2.03image/s]


Epoch_file: epoch_400. Total images: 1656, predicted_correct': 1141, predicted_correct (%)': 68.90096618357488%


epoch_396: Processing images: 100%|██████████| 1656/1656 [13:27<00:00,  2.05image/s]


Epoch_file: epoch_396. Total images: 1656, predicted_correct': 1130, predicted_correct (%)': 68.23671497584542%


epoch_392: Processing images: 100%|██████████| 1656/1656 [13:46<00:00,  2.00image/s]


...

Epoch_file: epoch_12. Total images: 1656, predicted_correct': 1047, predicted_correct (%)': 63.22463768115942%


epoch_8: Processing images: 100%|██████████| 1656/1656 [13:33<00:00,  2.04image/s]


Epoch_file: epoch_8. Total images: 1656, predicted_correct': 891, predicted_correct (%)': 53.80434782608695%


epoch_4: Processing images: 100%|██████████| 1656/1656 [13:53<00:00,  1.99image/s]

Epoch_file: epoch_4. Total images: 1656, predicted_correct': 687, predicted_correct (%)': 41.48550724637681%
